# Optimal stopping criteria for self consistency

How can we find the optimal stopping criteria for self consistency.

I believe the optimal strategy is to find if the probability of the most repeated answer is higher
than the probability of the second most repeated answer. If we can say that the difference is significative
for some confidence level then it does not have sense to sample more answers.

This strategy should be better than previous approaches because:

- It is theoretically grounded
- It takes into account the second most repeated answer, previous methods didn't. The following examples are not the same: `[5, 5, 5, 3, 4]` and `[5, 5, 5, 4, 4]`, the confidence levels are 48% and 84%, so the difference is huge.

## Code

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import re
from IPython.display import display, Markdown
import json
from tqdm.auto import tqdm

from transformers import AutoTokenizer

plt.plot()
plt.close('all')
plt.rcParams["figure.figsize"] = (20, 5)
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['font.size'] = 16

In [ ]:
import logging
from scipy.stats import norm

for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')


def is_difference_significative(n_first, n_second, n_tries, confidence_level=0.85):
    if n_second == 0:
        # uncertainty estimation equations do not work with n_second = 0
        if n_first == n_tries:
            return is_difference_significative(n_first, 1, n_tries + 1, confidence_level)
        elif n_first < n_tries:
            return is_difference_significative(n_first, 1, n_tries, confidence_level)
        else:
            raise ValueError()
    p_first = n_first/n_tries
    p_second = n_second/n_tries
    diff_variance = (p_first*(1-p_first)/n_tries + p_second*(1-p_second)/n_tries)**0.5
    z = (p_first - p_second)/diff_variance
    logging.info(f'p_first: {p_first*100:.1f}% p_second: {p_second*100:.1f}% Confidence level for the difference: {2*(norm.cdf(z) - 0.5)*100:.1f}%')
    return z > norm.interval(confidence_level)[1]

is_difference_significative(4, 0, 4)

## Visualization against previous stopping criteria

In [ ]:
stopping_criterias = dict(sqrt=[])

for n_tries in range(1, 21):
    for n_first in range(1, n_tries + 1):
        if n_first > n_tries**0.5:
            stopping_criterias['sqrt'].append([n_tries, n_first])
            break

for confidence_level in [0.6, 0.8, 0.9, 0.95]:
    key = f'confidence_level_{confidence_level:.2f}'
    stopping_criterias[key] = []
    for n_tries in range(1, 21):
        for n_first in range(1, n_tries + 1):
            if is_difference_significative(n_first, 0, n_tries, confidence_level=confidence_level):
                stopping_criterias[key].append([n_tries, n_first])
                break

stopping_criterias

In [ ]:
offset = 0.04
for stopping_criteria, values in stopping_criterias.items():
    logging.info(f'{stopping_criteria}:')
    values = np.array(values)
    plt.plot(values[:, 0], values[:, 1] + offset, label=stopping_criteria)
    offset -= 0.04


plt.xlim(2, 20)
plt.xticks(range(2, 21))
plt.yticks(range(2, 6))
plt.legend()
plt.grid()
plt.xlabel('Number of tries')
plt.ylabel('Number of successes to stop')

The `sqrt` criteria is different to all the proposed confidence levels. At low number of tries it behaves like a confidence level of 60%, but at higher number of tries behaves like a 90% confidence level.

On all my previous experiments I was simply using a majority criteria threshold, f.e. for 8 tries I required 4 identical responses.
I believe the biggest advantage will come from devoting more compute to harder problems.

## TODO:

- [x] Better logging of results
- [x] Document different options and compare to previous implementations
- [x] Why this is good? It uses all the information, not just the most frequent answer
- [ ] Implement in code and compare speed and results